In [1]:
from datetime import date
import warnings
warnings.filterwarnings('ignore')

In [84]:
from datetime import date

chandlierMultiplier = 2
max_holding = 100
source = "CSV"

#dateTimeFormat = '%Y-%m-%d'
dateTimeFormat = '%d-%b-%Y'

# filter
days = 30 * 24

# frequency
freq = "1D"
symbol = 'ICICIBANK'
myDate="31_Mar_2019"
filename ="./data/"+symbol+"_"+myDate+".csv"

# Calculate indicators and fields
%run "freedom_indicators_calculation.ipynb"


########## Algorithm #############
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0
    
class Trend(Enum):
    SIDE = 0
    UP = 1
    SUP = 2
    DOWN = 3
    SDOWN = 4

In [85]:
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR

# Math
price['priceSD'] = STDDEV(price.close)
price['priceVAR'] = VAR(price.close)

price['delBBUP'] = (price.bbup - price.close)/(price.bbup-price.bbmid)
price['delBBLOW'] = (price.close - price.bblow)/(price.bbmid-price.bblow)

price['d1Slope'] = np.arctan(price.close - price.close.shift(1)) * 180 / np.pi
price['d3Slope'] = np.arctan(price.close - price.close.shift(3)) * 180 / np.pi
price['d5Slope'] = np.arctan(price.close - price.close.shift(5)) * 180 / np.pi
price['d10Slope'] = np.arctan(price.bbmid - price.bbmid.shift(3)) * 180 / np.pi
price['d15Slope'] = np.arctan(price.close - price.close.shift(15)) * 180 / np.pi

price['macd1dSlope'] = np.arctan(price.macd - price.macd.shift(1)) * 180 / np.pi

price['macdTSF'] = TSF(price.macd)
price['priceTSF'] = TSF(price.close)


TrendThreshold = 85
trend = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
trend[price.d10Slope >= TrendThreshold] = Trend.UP
trend[price.d10Slope <= -1 * TrendThreshold] = Trend.DOWN
#price.trend1D[price1dGradient >= 65] = Trend.SUP
#price.trend1D[price1dGradient <= -65] = Trend.SDOWN

price['trend1D'] = trend

price['sdHighLow'] = STDDEV((price.high - price.close))
price['varHighLow'] = STDDEV((price.high - price.close))
price['delPriceHighLow'] = (price.high - price.close)/price.sdHighLow

price['PriceChange%'] = (price.close - price.shift(1).close)*100/price.close

price.dropna(inplace=True)
labels = pd.DataFrame(data = {"holdings":np.full(price.index.shape[0], S.HOLD)},index=price.index)
#Entry Criterion
labels[
    (price['trend1D'] == Trend.UP)
] = S.BUY

#Exit Criterion
labels[
     (price['trend1D'] == Trend.DOWN)
] = S.SELL


# buy and sell decisions are stored in arrays for plotting purpose(visualisation)

buy = price[labels['holdings']==S.BUY]
sell = price[labels['holdings']==S.SELL]

#################################
#   Plotting Data              #
###############################

rangeMin = price.index[-150]
rangeMax = price.index[-1]

%run "freedom_plot.ipynb"

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]
[ (7,1) x1,y7 ]
[ (8,1) x1,y8 ]
[ (9,1) x1,y9 ]



## Entry Criterion
- MACD crossover when market is trending up
- BB when market is moving sideways

## Exit Criterion
- Stochastics

## Identify market trend
- low ATR means market moving sideways
- Flat(gradient=0) MACD means market is moving sideways
- low ATR indicates low volatility - market is moving sideways
- High ATR indicates high volatility - market is trending
- Gradient of current - last 15 days


ADX Value	Trend Strength
- 0-25	Absent or Weak Trend
- 25-50	Strong Trend
- 50-75	Very Strong Trend
- 75-100	Extremely Strong Trend